In [1]:
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import os
import pytz
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException

In [2]:
def init_driver(headless, proxy=None, chrome_driver_path=None):
    """Initialize an instance of Chrome Driver"""
    chrome_options = webdriver.ChromeOptions()
    if headless is True:
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--headless')
    if proxy is not None:
        chrome_options.add_argument(f'--proxy-server={proxy}')

    chrome_options.add_argument("--start-maximized")
    chrome_options.arguments.extend(["--no-sandbox", "--disable-setuid-sandbox", "--disable-dev-shm-usage", "javascript.enabled"])
#     chrome_options.add_argument(f'--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
#     chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
#     chrome_options.add_experimental_option('useAutomationExtension', False)

    service = Service(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

chrome_driver_path = r"C:\Users\rishita.mathur\Downloads\chromedriver_win32\chromedriver.exe"
driver = init_driver(False, chrome_driver_path=chrome_driver_path)

In [3]:
# url = 'https://www.trustpilot.com/review/mclhomemade.com'
url = 'https://www.trustpilot.com/review/www.sprinly.com'

In [4]:
def get_details(review):
    try:
        Title = review.find_element(By.XPATH, value = ".//h2[@class='typography_heading-s__f7029 typography_appearance-default__AAY17']").text
    except:
        return
    try:
        Customer_name = review.find_element(By.XPATH, value = ".//span[@class='typography_heading-xxs__QKBS8 typography_appearance-default__AAY17']").text
    except:
        return
    try:
        Review_rating = str(review.find_element(By.XPATH, value=".//*[@class='star-rating_starRating__4rrcf star-rating_medium__iN6Ty']//img").get_attribute("alt").split(" ")[1])
#         Review_rating = str(review.find_element(By.XPATH, value=".//*[@class='star-rating_starRating__4rrcf star-rating_medium__iN6Ty']//img").get_attribute("alt").split(" ")[1]) + "/" + str(review.find_element(By.XPATH, value=".//*[@class='star-rating_starRating__4rrcf star-rating_medium__iN6Ty']//img").get_attribute("alt").split(" ")[4])
        print(Review_rating)
    except:
        Review_rating = 0
            
#     try:
#         Review_date = review.find_element(By.XPATH, value = ".//*[(@class='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH')]//datetime").get_attribute("time").text
#     except:
#         Review_date = "Not Found"
    try:
        Review_date = review.find_element(By.XPATH, value=".//div[@class='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH']/time").text
    except:
        Review_date = "Not Found"    
    try:
        Date_of_experience = review.find_element(By.XPATH, value = ".//p[contains(@class, 'typography_body-m__xgxZ_ typography_appearance-default__AAY17')]").text.split(':')[1]
    except:
        return
    try:
        Review_Text = review.find_element(By.XPATH, value = ".//p[@class='typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn']").text.replace("\n", " ")
    except:
        Review_Text = ""
    try:
        Restaurant_name = driver.find_element(By.XPATH, value=".//a[@class='link_internal__7XN06 link_disabled__mIxH1 typography_body-s__aY15Q typography_appearance-subtle__8_H2l link_link__IZzHN link_notUnderlined__szqki breadcrumb_breadcrumbLink__p1PMo']").text
    except:
        Restaurant_name = ""
    try:
        Address = driver.find_element(By.XPATH, value=".//ul[@class='typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_contactInfoAddressList__RxiJI']")
        Address = Address.text
    except:
        Address = ""
    try:
        Customer_id_url = review.find_element(By.XPATH, value=".//div[@class='styles_consumerDetailsWrapper__p2wdr']//a").get_attribute("href")
    except:
        Customer_id_url = ""
        
    Location= Restaurant_name + '\n' +Address  
    
    try:
        responded_flag = review.find_element(By.XPATH, './/p[@class="typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX"]').text
    except:
        responded_flag = ""

    if responded_flag:
        already_responded_flag = "TRUE"
    else:
        already_responded_flag = "FALSE"
    
    try:
#         Location_url = 'https://www.trustpilot.com/review/mclhomemade.com'  
        Location_url = 'https://www.trustpilot.com/review/www.sprinly.com'
    except:
        Location_url = ""
    
#     details = (Title, Customer_name, Review_rating, Date_of_experience, Review_Text, Restaurant_name, Address, Location, Customer_id_url, Location_url)
    details = (Location, Customer_name, Review_rating, Review_date, Date_of_experience, Customer_id_url, Review_Text, already_responded_flag, Location_url)
#     details = (Location, Customer_name, Review_rating, Date_of_experience, Customer_id_url, Review_Text, Location_url)
    return details


driver.get(url)
time.sleep(random.randint(6,8))

## The button accepts cookies
button = driver.find_element(By.XPATH,value = ".//button[@id='onetrust-accept-btn-handler']")
button.click()

data = []
while True:
    try:
        reviews = driver.find_elements(By.XPATH, value = ".//div[@class='styles_reviewCardInner__EwDq2']")
        print(len(reviews))
    except:
        print('No Review available')
        break

    for review in reviews:
        details = get_details(review)
        data.append(details)

    
    try:
        next_button = driver.find_element(By.XPATH, value = ".//a[@class='link_internal__7XN06 button_button__T34Lr button_m__lq0nA button_appearance-outline__vYcdF button_squared__21GoE link_button___108l pagination-link_next__SDNU4 pagination-link_rel__VElFy']")
        next_button.click()
        time.sleep(10)
    except NoSuchElementException:
        next_button = None
        print("Reviews Completed.")
        break

        


# df = pd.DataFrame(data, columns = ['Title', 'Customer_name', 'Review_rating', 'Date_of_experience', 'Text', 'Restaurant_name','Address','Location','Customer_id_url', 'Location_url'])
df = pd.DataFrame(data, columns = ['Location', 'Customer_name', 'Review_rating', 'Review_date', 'Date_of_experience',  'Customer_id_url', 'Review_Text', 'already_responded_flag','Location_url'])

# # Get the URL of the current page
# current_url = driver.current_url

# # Create a new column in the DataFrame called "Location_url"
# df["Location_url"] = current_url 

try:
    df.to_csv('Trustpilot_Test2.csv', index=False)
    print("CSV file generated successfully.")
except Exception as e:
    print("An error occurred while generating the CSV file:")
    print(str(e))

# df.to_csv('Trustpilot_Test.csv', index = False)

20
1
5
5
1
5
5
5
5
5
5
1
5
1
4
5
1
5
5
5
5
20
5
3
5
5
5
5
5
5
5
5
4
5
5
5
4
5
4
5
5
5
20
5
5
4
5
5
1
5
4
3
5
5
4
5
5
5
3
2
4
5
5
20
1
5
4
5
5
5
3
2
5
2
5
5
1
4
5
5
5
4
5
5
20
5
4
5
4
5
5
5
5
3
5
5
5
2
5
2
5
3
5
5
5
20
1
5
5
5
1
5
1
4
3
1
1
3
5
4
4
1
4
3
5
5
20
4
3
5
5
3
5
3
3
3
2
5
3
3
4
3
5
5
5
5
3
20
3
3
1
5
5
4
4
4
4
3
4
5
3
5
5
5
5
5
3
1
20
1
5
4
5
4
5
3
5
3
5
5
5
5
5
5
5
4
2
4
1
20
5
5
5
5
5
5
4
5
4
5
4
1
5
1
5
5
5
5
5
4
20
5
4
2
5
5
2
1
5
5
2
5
4
3
5
5
5
5
5
5
5
20
2
5
5
5
5
5
1
5
4
5
5
1
5
4
5
4
4
3
5
4
20
4
5
5
1
5
4
5
1
1
4
5
5
2
4
4
5
5
5
5
4
20
5
5
4
3
5
5
4
5
5
1
5
3
1
5
5
5
5
4
5
5
20
5
5
5
4
1
4
5
5
5
5
5
1
5
3
5
5
5
4
5
3
20
5
5
4
3
5
5
4
5
4
4
4
5
1
2
5
5
3
5
5
1
20
5
5
4
2
1
5
2
3
5
1
1
1
5
5
5
4
5
1
5
5
20
3
5
1
5
5
5
5
4
1
1
2
5
2
1
5
5
1
1
1
4
20
4
5
1
3
5
1
2
3
5
5
1
5
4
4
1
1
4
1
1
3
8
1
2
1
2
5
5
3
1
Reviews Completed.
CSV file generated successfully.


In [5]:
df_ii = pd.read_csv('Trustpilot_Test_Success.csv')

In [ ]:
df.to_csv('C:\\Users\\rishita.mathur\\Documents\\PyCharm Projects\\Trustpilot', index = False)

In [ ]:
df_ii

In [ ]:
print(os.getcwd())

In [ ]:
driver.quit()

In [ ]:
df

In [ ]:
with open('Trustpilot_Test.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    for row in csv_reader:
        print(row)